<a href="https://colab.research.google.com/github/AnzorGozalishvili/NASA_ODSR_DATA/blob/main/NASA_OSDR_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [3]:
import datasets

In [4]:
from datasets import load_dataset

In [9]:
dataset = load_dataset('anz2/NASA_OSDR')

In [17]:
dataset['train'][10]

{'Sample Name': 'GSM2684068',
 'Protocol REF': 'Nucleic Acid Extraction',
 'Parameter Value: DNA Fragmentation': 'sonication',
 'Parameter Value: DNA Fragment Size': '200-300 base pair',
 'Extract Name': 'GSM2684068',
 'Protocol REF.1': 'Library Construction',
 'Parameter Value: Library Strategy': 'BisPCR2',
 'Parameter Value: Library Selection': 'other',
 'Parameter Value: Library Layout': 'PAIRED',
 'Protocol REF.2': 'Nucleic Acid Sequencing',
 'Parameter Value: Sequencing Instrument': 'Illumina MiSeq',
 'Assay Name': 'BisPCR2',
 'Parameter Value: Read Length': '150 base pair',
 'Raw Data File': 'GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz, GLDS-524_wgbs_GSM2684068_R2_raw.fastq.gz',
 'Protocol REF.3': 'GeneLab raw data processing protocol',
 'Parameter Value: Read Depth': '199027 read',
 'Parameter Value: MultiQC File Names': 'GLDS-524_Gwgbs_raw_multiqc_report.zip'}

In [18]:
fasta_file_names = dataset['train'][10]['Raw Data File']

In [19]:
fasta_file_names

'GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz, GLDS-524_wgbs_GSM2684068_R2_raw.fastq.gz'

In [ ]:
!pip install boto3

In [38]:
import boto3
import os
from botocore import UNSIGNED
from botocore.client import Config

In [39]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [43]:
NASA_OSDR_BUCKET = 'nasa-osdr'
DOWNLOAD_DIR = 'nasa_osdr_fasta_files'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [53]:
def download_fasta_file_from_s3(fasta_filename):
    response = s3.list_objects_v2(Bucket=NASA_OSDR_BUCKET, Prefix='OSD-524')

    if 'Contents' in response:
        for obj in response['Contents']:
            key = obj['Key']
            if key.endswith(fasta_filename):
                file_path = os.path.join(DOWNLOAD_DIR, fasta_filename)
                s3.download_file(Bucket=NASA_OSDR_BUCKET, Key=key, Filename=file_path)
                print('file downloaded to path:', file_path)

In [45]:
fasta_file_names.split(',')[0]

'GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz'

In [54]:
download_fasta_file_from_s3(fasta_file_names.split(',')[0])

file downloaded to path: nasa_osdr_fasta_files/GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz
file downloaded to path: nasa_osdr_fasta_files/GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz


In [55]:
!ls nasa_osdr_fasta_files

GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz


In [58]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.7 MB/s eta 0:00:00


In [66]:
from Bio import SeqIO
import gzip

In [79]:
def load_rna_sequences_from_fasta(fasta_file):
    with gzip.open(fasta_file, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            yield record

In [80]:
fasta_loader = load_rna_sequences_from_fasta('nasa_osdr_fasta_files/GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz')
sample = next(iter(fasta_loader))

In [82]:
sample.__dict__

{'_seq': Seq('GTTTTTTTTAAAGTGTTAGATAGTAGTTAAAATAAGAGATATGTTTTTTATTTA...AGG'),
 'id': 'SRR5756273.1',
 'name': 'SRR5756273.1',
 'description': 'SRR5756273.1 1 length=151',
 'dbxrefs': [],
 'annotations': {},
 '_per_letter_annotations': {'phred_quality': [32,
   32,
   32,
   32,
   32,
   32,
   35,
   35,
   35,
   31,
   16,
   16,
   35,
   38,
   35,
   38,
   38,
   37,
   38,
   37,
   38,
   34,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   37,
   38,
   37,
   37,
   37,
   33,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   36,
   35,
   37,
   38,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   38,
   38,
   38,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   37,
   32,
   38,
   34,
   30,
   36,
   39,


In [83]:
!git clone https://github.com/mana438/RNABERT.git

Cloning into 'RNABERT'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 80 (delta 31), reused 66 (delta 17), pack-reused 0
Receiving objects: 100% (80/80), 347.43 KiB | 3.66 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [84]:
!apt-get install -y g++

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [88]:
!apt-get install build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [87]:
!cd RNABERT && python setup.py install

/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        S